# I- DESCRIPTION

In [ ]:
qMRinfo('mwf'); % Describe the model

# II- MODEL PARAMETERS
## a- Create object

In [ ]:
Model = mwf;

## Download sample data from OSF
> The current `Model` is an instance of `mwf` class.
 
You can manually download the sample data for `mwf` [by clicking here](https://osf.io/bes6f/download?version=3).

In [ ]:
dataDir = downloadData(Model,pwd);

## b- Set protocol
> Protocol is set according to the example data
 
%          |- mwf object needs 1 protocol field(s) to be assigned:
%          |-   MET2data

In [ ]:
EchoTime  = [10.0000; 20.0000; 30.0000; 40.0000; 50.0000; 60.0000; 70.0000; 80.0000; 90.0000; 100.0000; 110.0000; 120.0000; 130.0000; 140.0000; 150.0000; 160.0000; 170.0000; 180.0000; 190.0000; 200.0000; 210.0000; 220.0000; 230.0000; 240.0000; 250.0000; 260.0000; 270.0000; 280.0000; 290.0000; 300.0000; 310.0000; 320.0000];
% EchoTime (ms) is a vector of [32X1]
Model.Prot.MET2data.Mat = [ EchoTime ];
%%   

# III- FIT EXPERIMENTAL DATASET
## a- Load experimental data
%          |- mwf object needs 2 data input(s) to be assigned:
%          |-   MET2data
%          |-   Mask

In [ ]:
 
% MET2data.mat contains [64  64   1  32] data.
 load('mwf_data/MET2data.mat');
% Mask.mat contains [64  64] data.
 load('mwf_data/Mask.mat');
 data.MET2data= double(MET2data);
 data.Mask= double(Mask);

## b- Fit experimental data
> This section will fit data.

In [ ]:
FitResults = FitData(data,Model,0);

## c- Show fitting results
> * Output map will be displayed.
 
> * If available, a graph will be displayed to show fitting in a voxel.

In [ ]:
qMRshowOutput(FitResults,data,Model);

## d- Save results
> * qMR maps are saved in NIFTI and in a structure `FitResults.mat` that can be loaded in qMRLab graphical user interface.
> * Model object stores all the options and protocol
> * These objects can be easily shared or be used for simulation.

In [ ]:
FitResultsSave_nii(FitResults);

# IV- SIMULATIONS
> This section can be executed to run simulations for vfa_t1.
 
## a- Single Voxel Curve
> Simulates single voxel curves:
 
       1. Use equation to generate synthetic MRI data
       2. Add rician noise
       3. Fit and plot curve

In [ ]:
      x = struct;
      x.MWF = 50.0001;
      x.T2MW = 20.0001;
      x.T2IEW = 120;
      % Set simulation options
      Opt.SNR = 200;
      Opt.T2Spectrumvariance_Myelin = 5;
      Opt.T2Spectrumvariance_IEIntraExtracellularWater = 20;
      % run simulation
      figure('Name','Single Voxel Curve Simulation');
      FitResult = Model.Sim_Single_Voxel_Curve(x,Opt);

## b- Sensitivity analysis
> Simulates sensitivity to fitted parameters: 
 
       1. Vary fitting parameters from lower (lb) to upper (ub) bound.
       2. Run Sim_Single_Voxel_Curve Nofruns times
       3. Compute mean and std across runs

In [ ]:
      %              MWF           T2MW          T2IEW         
      OptTable.st = [50            20            1.2e+02]; % nominal values
      OptTable.fx = [0             1             1]; %vary MWF...
      OptTable.lb = [0.0001        0.0001        40]; %...from 0.0001
      OptTable.ub = [1e+02         40            2e+02]; %...to 100
      % Set simulation options
      Opt.SNR = 200;
      Opt.T2Spectrumvariance_Myelin = 5;
      Opt.T2Spectrumvariance_IEIntraExtracellularWater = 20;
      Opt.Nofrun = 5;
      % run simulation
      SimResults = Model.Sim_Sensitivity_Analysis(OptTable,Opt);
      figure('Name','Sensitivity Analysis');
      SimVaryPlot(SimResults, 'MWF' ,'MWF' );